This notebook holds a pytorch CNN that classifies the cell crops into their cell cycle phase with 4 channels

In [1]:
import pandas as pd
import numpy as np
from ccc_nn_functions import str2array

csv_file = r'C:\Users\rz200\Documents\development\cell-SCT\classification\imported_CSV\dataframe_821'
df = pd.read_csv(csv_file)

In [3]:
import sys
#source:https://stackoverflow.com/questions/3160699/python-progress-bar
def progressbar(it, prefix="", size=60, out=sys.stdout): # Python3.3+
    count = len(it)
    def show(j):
        x = int(size*j/count)
        print("{}[{}{}] {}/{}".format(prefix, "#"*x, "."*(size-x), j, count),
              end='\r', file=out, flush=True)
    show(0)
    for i, item in enumerate(it):
        yield item
        show(i+1)
    print("\n", flush=True, file=out)

def df_ignore_rows(df):
    #indices_to_skip_img_wrong_shape = [i for i in range(len(df)) if str2array(df['pcna_crops'][i]).dtype is np.dtype('object')]  # skipping rows with shapes such as (7,)

    indices_to_skip_img_wrong_shape = []
    for i in progressbar(range(len(df))):
        if str2array(df['pcna_crops'][i]).dtype is np.dtype('object'):
            indices_to_skip_img_wrong_shape.append(i)


    indices_to_skip_no_class = df[(df['G1_Phase'] == False) & (df['S_Phase'] == False) & (df['G2_M_Phase'] == False)].index

    rows_to_ignore = np.concatenate((indices_to_skip_img_wrong_shape, indices_to_skip_no_class), axis=0)
    df = df.drop(set(rows_to_ignore)).reset_index(drop=True) #dropping the rows to ignore

    return df

df = df_ignore_rows(df)

C:\Users\rz200\Documents\development\cell-SCT\classification\ccc_nn_functions.py:106: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(ast.literal_eval(s))


[############################################################] 140034/140034



In [79]:
def get_crops(df):
    dapi_crops = []
    cyclina2_crops = []
    edu_crops = []
    pcna_crops = []
    for i in progressbar(range(len(df)-110000)):
        dapi_crops.append(str2array(df['dapi_crops'][i])*255)
        cyclina2_crops.append(str2array(df['cyclina2_crops'][i])*255)
        edu_crops.append(str2array(df['edu_crops'][i])*255)
        pcna_crops.append(str2array(df['pcna_crops'][i])*255)
    dapi_crops = np.array(dapi_crops)
    cyclina2_crops = np.array(cyclina2_crops)
    edu_crops = np.array(edu_crops)
    pcna_crops = np.array(pcna_crops)

    all_crops = np.dstack((dapi_crops,cyclina2_crops,edu_crops,pcna_crops)).squeeze()
    return all_crops

In [80]:
crops = get_crops(df)

[############################################################] 16486/16486



C:\Users\rz200\AppData\Local\Temp\ipykernel_21200\2756014679.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dapi_crops = np.array(dapi_crops)
C:\Users\rz200\AppData\Local\Temp\ipykernel_21200\2756014679.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cyclina2_crops = np.array(cyclina2_crops)
C:\Users\rz200\AppData\Local\Temp\ipykernel_21200\2756014679.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If

In [76]:
from PIL import Image
img_id = 1000
arr_dapi = str2array(df['dapi_crops'][img_id])*255
arr_cyclina2 = str2array(df['cyclina2_crops'][img_id])*255
arr_edu = str2array(df['edu_crops'][img_id])*255
arr_pcna = str2array(df['pcna_crops'][img_id])*255

arr = np.dstack((arr_dapi,arr_cyclina2,arr_edu,arr_pcna))
arr = arr.astype(np.uint8)

print(arr)

im = Image.fromarray(arr)

[[[ 1 31  1 37]
  [ 1 31  1 38]
  [ 1 30  1 38]
  ...
  [ 1 31  1 39]
  [ 1 31  1 40]
  [ 1 32  1 41]]

 [[ 1 30  1 39]
  [ 1 31  1 38]
  [ 1 30  0 39]
  ...
  [ 1 32  1 40]
  [ 0 32  1 40]
  [ 1 33  1 41]]

 [[ 1 31  1 38]
  [ 1 31  0 38]
  [ 0 30  1 39]
  ...
  [ 1 31  1 39]
  [ 1 31  1 41]
  [ 1 33  1 40]]

 ...

 [[ 1 31  1 38]
  [ 1 30  1 38]
  [ 1 31  1 40]
  ...
  [ 1 31  1 41]
  [ 1 31  1 41]
  [ 1 31  1 40]]

 [[ 1 30  1 38]
  [ 1 31  1 38]
  [ 1 30  1 39]
  ...
  [ 1 32  1 40]
  [ 1 31  1 40]
  [ 1 32  1 41]]

 [[ 1 29  1 38]
  [ 1 29  1 38]
  [ 1 31  1 40]
  ...
  [ 1 31  1 40]
  [ 1 31  1 40]
  [ 1 31  1 41]]]


In [75]:


image = crops[0]*255

image_to_np = []
for arr in image:
    image_to_np.append(np.array(arr))
image = np.array(image_to_np)

#image = np.expand_dims(image, axis=0)

image = image.astype(np.uint8)
print(image)
image = Image.fromarray(image)

[[[ 1  0  1 ...  1  1  1]
  [ 1  1  1 ...  1  1  1]
  [ 1  1  1 ...  1  1  1]
  ...
  [ 1  1  2 ...  2  2  1]
  [ 1  2  1 ...  2  2  1]
  [ 1  1  2 ...  2  2  2]]

 [[15 13 13 ... 15 14 13]
  [15 14 13 ... 17 16 16]
  [14 15 15 ... 18 18 16]
  ...
  [19 20 20 ... 17 16 17]
  [19 21 21 ... 17 17 17]
  [21 20 22 ... 16 18 18]]

 [[ 0  0  1 ...  1  1  0]
  [ 1  1  1 ...  0  1  0]
  [ 0  0  0 ...  1  0  1]
  ...
  [ 1  1  1 ...  1  1  1]
  [ 1  1  1 ...  1  1  1]
  [ 1  1  1 ...  1  1  1]]

 [[20 21 22 ... 22 22 22]
  [22 22 22 ... 21 23 21]
  [23 21 24 ... 25 22 22]
  ...
  [26 25 25 ... 25 24 24]
  [26 24 24 ... 24 26 25]
  [24 26 28 ... 24 25 27]]]


TypeError: Cannot handle this data type: (1, 1, 22), |u1

In [19]:
def get_cell_labels(df):
    g1_indices = df[(df['G1_Phase'] == True)].index
    s_indices = df[(df['S_Phase'] == True)].index
    g2_m_indices = df[(df['G2_M_Phase'] == True)].index

    #make an array that is the length of all of these indices put into one, that is made of 0s
    #replace the 0s accordingly by which phase index they correspond to

    cell_labels = np.arange(len(g1_indices)+len(s_indices)+len(g2_m_indices))

    np.put(cell_labels,g1_indices,np.zeros(len(g1_indices)))
    np.put(cell_labels,s_indices,np.ones(len(s_indices)))
    np.put(cell_labels,g2_m_indices,np.full(len(g2_m_indices),2))

    return cell_labels

cell_labels = get_cell_labels(df)

In [81]:
import torchvision

class CenterCrop(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        image = torchvision.transforms.CenterCrop(self.output_size).forward(image)
        return {'image': image, 'label': label}

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, label = sample['image'], sample['label']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C x H x W
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image),
                'label': torch.tensor(label)}

In [85]:
import torch
from torch.utils.data.dataset import Dataset
from PIL import Image

class CellDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, crops, labels, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.crops = crops
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = self.crops[idx]


        image_to_np = []
        for arr in image:
            image_to_np.append(np.array(arr))
        image = np.array(image_to_np)

        #image = np.expand_dims(image, axis=0)

        image = image.astype(np.uint8)
        #print(image)
        image = Image.fromarray(image)

        label = self.labels[idx]

        sample = {'image': image, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [86]:
from torchvision.transforms import transforms

transformed_dataset = CellDataset(crops=crops,
                                        labels=cell_labels,
                                        transform=transforms.Compose([
                                            CenterCrop(32),
                                            ToTensor()]))

In [87]:
img,label = transformed_dataset.__getitem__(0)

TypeError: Cannot handle this data type: (1, 1, 22), |u1